## set running directory

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=True)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

In [6]:
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

Please create a subscriber to the marker


## init planning scene

In [7]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## ui

In [8]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [209]:
from pkg.utils.gjk import get_point_list, get_gjk_distance

In [338]:
##
# @class ObstacleBase
# @brief base class for obstacle generators
class ObstacleBase:
    RTH_MIN = None ## R: center ~ nearest point
    RTH_MAX = None
    RPY_MIN = None
    RPY_MAX = None
    DIM_MIN = None
    DIM_MAX = None
    GTYPE = None
    COLOR = (0.7,0.7,0.7,1)
    
    def __init__(self, gscene, name, sampler=np.random.uniform, DIM=None, RTH=None, RPY=None):
        self.name = name
        self.DIM = sampler(self.DIM_MIN, self.DIM_MAX) if DIM is None else DIM
        self.RTH = sampler(self.RTH_MIN, self.RTH_MAX) if RTH is None else RTH
        self.RPY = sampler(self.RPY_MIN, self.RPY_MAX) if RPY is None else RPY
        self.RPY[2] += self.RTH[1]
        self.XYZ = np.array(cyl2cart(*self.RTH))
        print("DIM: {}".format(self.DIM))
        print("RTH: {}".format(self.RTH))
        print("XYZ: {}".format(self.XYZ))
        print("RPY: {}".format(self.RPY))
        verts_rotated = np.matmul(Rot_rpy(self.RPY), (DEFAULT_VERT_DICT[self.GTYPE]*self.DIM).transpose())
        print("verts_rotated:\n {}".format(verts_rotated.transpose()))
        xy_normed = self.XYZ[:2]/(np.linalg.norm(self.XYZ[:2])+1e-6)
        print("xy_normed: {}".format(xy_normed))
        verts_r_compo = np.dot(xy_normed, verts_rotated[:2,:])
        print("verts_r_compo: {}".format(verts_r_compo))
        self.XYZ[:2] -= xy_normed[:2]*np.min(verts_r_compo)
        self.RTH[0] -= np.min(verts_r_compo)
        print("XYZ: {}".format(self.XYZ))
        self.geometry = gscene.create_safe(gtype=self.GTYPE, name=self.name, link_name="base_link", 
                                  dims=self.DIM, center=tuple(self.XYZ), rpy=self.RPY,
                                  color=self.COLOR, display=True, collision=True, fixed=True)
        self.subgeo_list = []
        
##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.8, 0.8, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,0.5)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.3
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-2
        
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, -np.pi/2)
    RPY_MAX = (0, 0, +np.pi/2)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, +0.0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Ceiling
# @brief Ceiling - upperbound of the workspace
class Ceiling(ObstacleBase):
    RTH_MIN = (0.0, 0, 1.0)
    RTH_MAX = (0.0, 0, 2)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Wall
# @brief define horizontal boundary of the workspace
class Wall(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (2.0, np.pi, 0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 6, 3)
    DIM_MAX = (0.1, 6, 3)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.8, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.3, 0.3, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class Bar
# @brief occasional poles that obstruct robot motion
class Bar(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.8, +np.pi, 1.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 4, 0.1)
    DIM_MAX = (0.3, 4, 0.3)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.02, 0.2, 0.2)
    DIM_MAX = (0.06, 0.2, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.2,0.8,0.5)
    def __init__(self, gscene, name, workplane, XYZ_LOC=None, **kwargs):
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        print("verts_rot_loc")
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)

WORKPLANE_TYPES = [WorkPlane, Box, SideBox, TopBox]

Nmax_wall, Nmax_pole, Nmax_bar = 4, 10, 10

## add floor, ceiling

In [339]:
floor = Floor(gscene, "floor")
# ceiling = Ceiling(gscene, "ceiling")

DIM: [3.  3.  0.1]
RTH: [ 0.          0.         -0.11535314]
XYZ: [ 0.          0.         -0.11535314]
RPY: [0. 0. 0.]
verts_rotated:
 [[-1.5  -1.5  -0.05]
 [-1.5  -1.5   0.05]
 [-1.5   1.5  -0.05]
 [-1.5   1.5   0.05]
 [ 1.5  -1.5  -0.05]
 [ 1.5  -1.5   0.05]
 [ 1.5   1.5  -0.05]
 [ 1.5   1.5   0.05]]
xy_normed: [0. 0.]
verts_r_compo: [0. 0. 0. 0. 0. 0. 0. 0.]
XYZ: [ 0.          0.         -0.11535314]
DIM: [3.  3.  0.1]
RTH: [0.         0.         1.28668891]
XYZ: [0.         0.         1.28668891]
RPY: [0. 0. 0.]
verts_rotated:
 [[-1.5  -1.5  -0.05]
 [-1.5  -1.5   0.05]
 [-1.5   1.5  -0.05]
 [-1.5   1.5   0.05]
 [ 1.5  -1.5  -0.05]
 [ 1.5  -1.5   0.05]
 [ 1.5   1.5  -0.05]
 [ 1.5   1.5   0.05]]
xy_normed: [0. 0.]
verts_r_compo: [0. 0. 0. 0. 0. 0. 0. 0.]
XYZ: [0.         0.         1.28668891]


## set workplane

In [340]:
wp = random.choice(WORKPLANE_TYPES)(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)

DIM: [0.39216963 0.53403892 0.05      ]
RTH: [ 0.78163687 -0.86534777  0.26913727]
XYZ: [ 0.50679411 -0.59507641  0.26913727]
RPY: [ 0.          0.         -1.80096007]
verts_rotated:
 [[-0.21524371  0.25183088 -0.025     ]
 [-0.21524371  0.25183088  0.025     ]
 [ 0.3047121   0.12999686 -0.025     ]
 [ 0.3047121   0.12999686  0.025     ]
 [-0.3047121  -0.12999686 -0.025     ]
 [-0.3047121  -0.12999686  0.025     ]
 [ 0.21524371 -0.25183088 -0.025     ]
 [ 0.21524371 -0.25183088  0.025     ]]
xy_normed: [ 0.64837456 -0.76131982]
verts_r_compo: [-0.33128239 -0.33128239  0.09859838  0.09859838 -0.09859838 -0.09859838
  0.33128239  0.33128239]
XYZ: [ 0.72158918 -0.84728826  0.26913727]


## add walls 

In [358]:
for iw in range(100):
    gname = "wl_{}".format(iw)
    if gname in gscene.NAME_DICT:
        gscene.remove(gscene.NAME_DICT[gname])
        
walls = []
for iw in range(np.random.choice(Nmax_wall)):
    wall = Wall(gscene, "wl_{}".format(iw))
    while wp.is_overlapped_with(wall.geometry):
        wall = Wall(gscene, "wl_{}".format(iw))
    walls.append(wall)

DIM: [0.1 6.  3. ]
RTH: [ 1.34246836 -0.81990987  0.        ]
XYZ: [ 0.91594885 -0.9814576   0.        ]
RPY: [ 0.          0.         -0.81990987]
verts_rotated:
 [[-2.22736737 -2.01030709 -1.5       ]
 [-2.22736737 -2.01030709  1.5       ]
 [ 2.15913866  2.08341552 -1.5       ]
 [ 2.15913866  2.08341552  1.5       ]
 [-2.15913866 -2.08341552 -1.5       ]
 [-2.15913866 -2.08341552  1.5       ]
 [ 2.22736737  2.01030709 -1.5       ]
 [ 2.22736737  2.01030709  1.5       ]]
xy_normed: [ 0.68228659 -0.73108379]
verts_r_compo: [-0.04999996 -0.04999996 -0.04999996 -0.04999996  0.04999996  0.04999996
  0.04999996  0.04999996]
XYZ: [ 0.95006315 -1.01801176  0.        ]
DIM: [0.1 6.  3. ]
RTH: [1.54339056 1.89938882 0.        ]
XYZ: [-0.49806934  1.4608153   0.        ]
RPY: [0.         0.         1.89938882]
verts_rotated:
 [[ 2.85562806  0.92080856 -1.5       ]
 [ 2.85562806  0.92080856  1.5       ]
 [-2.82335694 -1.01545831 -1.5       ]
 [-2.82335694 -1.01545831  1.5       ]
 [ 2.82335694  

## add pole

In [369]:
for iw in range(100):
    gname = "po_{}".format(iw)
    if gname in gscene.NAME_DICT:
        gscene.remove(gscene.NAME_DICT[gname])
    gname = "bar_{}".format(iw)
    if gname in gscene.NAME_DICT:
        gscene.remove(gscene.NAME_DICT[gname])

polls = []
for ip in range(np.random.choice(Nmax_pole)):
    pole = Pole(gscene, "po_{}".format(ip))
    while wp.is_overlapped_with(pole.geometry):
        pole = Pole(gscene, "po_{}".format(ip))
    polls.append(pole)

bars = []
for ib in range(np.random.choice(Nmax_bar)):
    bar = Bar(gscene, 'bar_{}'.format(ib))
    while wp.is_overlapped_with(bar.geometry):
        bar = Bar(gscene, 'bar_{}'.format(ib))
    bars.append(bar)

DIM: [0.18521203 0.22128849 4.        ]
RTH: [0.38372729 0.03845949 0.        ]
XYZ: [0.38344354 0.01475432 0.        ]
RPY: [ 0.         0.        -0.3577343]
verts_rotated:
 [[-0.12548577 -0.07121338 -2.        ]
 [-0.12548577 -0.07121338  2.        ]
 [-0.04800097  0.1360659  -2.        ]
 [-0.04800097  0.1360659   2.        ]
 [ 0.04800097 -0.1360659  -2.        ]
 [ 0.04800097 -0.1360659   2.        ]
 [ 0.12548577  0.07121338 -2.        ]
 [ 0.12548577  0.07121338  2.        ]]
xy_normed: [0.99925792 0.0384499 ]
verts_r_compo: [-0.12813079 -0.12813079 -0.04273362 -0.04273362  0.04273362  0.04273362
  0.12813079  0.12813079]
XYZ: [0.51147925 0.01968093 0.        ]
DIM: [0.22968177 0.17089303 4.        ]
RTH: [0.76542499 2.47293092 0.        ]
XYZ: [-0.60059226  0.47451487  0.        ]
RPY: [0.         0.         2.23349935]
verts_rotated:
 [[ 0.13801608 -0.03796179 -2.        ]
 [ 0.13801608 -0.03796179  2.        ]
 [ 0.00329563 -0.14310372 -2.        ]
 [ 0.00329563 -0.14310372 

## add object

In [344]:
obj = PlaneObject(gscene, "obj", wp)

DIM: [0.05964348 0.2        0.2       ]
RTH: [ 0.60829554 -1.30393469  0.15809292]
XYZ: [ 0.16041086 -0.58676385  0.15809292]
RPY: [ 0.          0.         -2.64425782]
verts_rotated:
 [[-0.02149944  0.10211322 -0.1       ]
 [-0.02149944  0.10211322  0.1       ]
 [ 0.07391755 -0.07365822 -0.1       ]
 [ 0.07391755 -0.07365822  0.1       ]
 [-0.07391755  0.07365822 -0.1       ]
 [-0.07391755  0.07365822  0.1       ]
 [ 0.02149944 -0.10211322 -0.1       ]
 [ 0.02149944 -0.10211322  0.1       ]]
xy_normed: [ 0.26370504 -0.96460166]
verts_r_compo: [-0.1041681  -0.1041681   0.09054327  0.09054327 -0.09054327 -0.09054327
  0.1041681   0.1041681 ]
XYZ: [ 0.18788051 -0.68724457  0.15809292]
verts_rot_loc


In [333]:
gscene.show_pose(crob.home_pose)

## dev conversion

In [87]:
from pkg.planning.filtering.lattice_predictor import *

In [88]:
gtimer = GlobalTimer.instance()
gtimer.reset()

## prepare latticizer

In [11]:
latticizer = Latticizer()

## prepare cells

In [12]:
gtimer.reset()
with gtimer.block("prepare_reference_cells"):
    latticizer.prepare_reference_cells(gscene)

## convert

In [13]:
with gtimer.block("convert"):
    latticizer.convert([gtem for gtem in gscene if gtem not in gtems_robot], crob.home_dict)

In [14]:
print(gtimer)

prepare_reference_cells: 	5902.0 ms/1 = 5902.221 ms (5902.221/5902.221)
convert: 	0.0 ms/1 = 0.007 ms (0.007/0.007)



In [15]:
gscene.show_pose(crob.home_pose)

In [16]:
# for coll_idxes in latticizer.coll_idx_dict.values():
#     for col_idx in coll_idxes:
#         gscene.copy_from(latticizer.cell_refs[col_idx])

## add brush

In [10]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,-0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## Register binders

In [12]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [13]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [14]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [15]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [16]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [17]:
sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [180]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

KeyError: 'indy0'

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## Test plan

In [20]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)

In [27]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=False)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = success
Remaining:4->3 / branching: 0->1 (0.85/300.0 s, steps/err: 7(25.6989002228 ms)/0.295332890961)
node: ('grip0', 0)->('grip0', 1) = success
Remaining:3->2 / branching: 1->2 (0.88/300.0 s, steps/err: 10(24.6050357819 ms)/1.50707682149)
node: ('grip0', 1)->('grip0', 2) = fail
node: ('grip0', 1)->('grip0', 2) = success
Remaining:2->1 / branching: 2->3 (1.04/300.0 s, steps/err: 29(33.6170196533 ms)/2.22764991945)
node: ('grip0', 2)->('goal', 2) = success
Remaining:1->0 / branching: 3->4 (1.05/300.0 s, steps/err: 6(17.3070430756 ms)/0.00162698225372)
======================= terminated 0: first answer acquired ===========================

## play searched plan

In [28]:
ppline.play_schedule(snode_schedule, period=0.01)

## deprecated python ver

In [ ]:
from pkg.utils.gjk import make_point3, get_point_list, get_point_list_list, get_point_list_list_from_point_list
from pkg.utils.gjk import get_gjk_distance, get_gjk_distance_min, get_gjk_distance_all

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("get_cell_vertices"):
    gscene_ref = GeometryScene(gscene.urdf_content, gscene.urdf_path, gscene.joint_names, gscene.link_names, rviz=False)
    centers = get_centers(Nwdh, L_CELL, OFFSET_ZERO).reshape((-1,3))
    cell_refs = []
    for icell, center in enumerate(centers):
        cell_refs.append(gscene_ref.create_safe(GEOTYPE.BOX, str(icell), "base_link", dims=(L_CELL,)*3, 
                                                center=center, rpy=(0,0,0), color=(1,1,1,0.2), 
                                                display=True, collision=False, fixed=True))
    cell_vertices = get_cell_vertices(centers, L_CELL)
    cell_vertices_gjk = get_point_list_list(cell_vertices)
    center_vertices_gjk = get_point_list_list(np.expand_dims(centers, axis=-2))

In [ ]:
print(gtimer)

In [ ]:
gtimer.reset()
sqrt3 = np.sqrt(3)
LCmax = L_CELL*sqrt3/2
Qdict = crob.home_dict
with gtimer.block("full"):
    for gtem in gscene:
#         if gtem in gtems_robot or not gtem.collision:
#             continue
        with gtimer.block("centers"):
            Tgtem = SE3_inv(gtem.get_tf(Qdict))
            centers_loc = np.abs(np.matmul(centers, Tgtem[:3,:3].transpose())+Tgtem[:3,3])
        if gtem.gtype == GEOTYPE.BOX and np.sum(np.abs(gtem.rpy))<1e-5:
            with gtimer.block("right box"):
                dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - L_CELL, axis=-1)
                cell_idx_occupy = np.where(dist_list < 1e-3)[0]
                continue
        with gtimer.block("calc_center_dist"):
            dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - LCmax, axis=-1)
            idx_candi = np.where(dist_list<0)[0]
        with gtimer.block("calc_points"):
            gtem_verts = gtem.get_vertice_from(Qdict)
        with gtimer.block("get_point_gjk"):
            gtem_verts_gjk = get_point_list(gtem_verts)
        with gtimer.block("get_distance_gjk"):
            cell_idx_occupy = []
            for idx in idx_candi:
                cell_candi = cell_vertices_gjk[idx]
                center_v_candi = center_vertices_gjk[idx]
                if ((get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem.radius < 1e-3)):
                    cell_idx_occupy.append(idx)
#                 if ((get_gjk_distance(gtem_verts_gjk, center_v_candi) - gtem.radius - L_CELL/2 < 1e-3) or 
#                     (get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem.radius < 1e-3)):
#                     cell_idx_occupy.append(idx)
print(gtimer)